<a href="https://colab.research.google.com/github/LeeSeungwon89/Deep-learning_Theory/blob/main/8-2%20%ED%95%A9%EC%84%B1%EA%B3%B1%20%EC%8B%A0%EA%B2%BD%EB%A7%9D%EC%9D%84%20%EC%82%AC%EC%9A%A9%ED%95%9C%20%EC%9D%B4%EB%AF%B8%EC%A7%80%20%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **8-2 합성곱 신경망을 사용한 이미지 분류**

## **패션 MNIST 데이터 불러오기**

## **합성곱 신경망 만들기**

## **모델 컴파일과 훈련**